In [1]:
# change the name of this variable to use a differnet dataset.
DATASET_NAME = 'gse129705'

In [2]:
# change these hyperparameters we search through.

# for graph
C_vals = [0.05 * i for i in range(1, 9)]
l1_ratios = [0.2 * i for i in range(1, 5)]


# for lasso
group_reg_lesso = [0.2  * i for i in range(1, 6)]
l1_reg_lasso = [0.05  * i for i in range(6)]

In [3]:
import matplotlib.pyplot as plt
import numpy as np

from group_lasso import LogisticGroupLasso


seed_value = 314159
np.random.seed(seed_value)

LogisticGroupLasso.LOG_LOSSES = True

In [4]:
import os
import math
import random
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import lmdb
import pickle
from torch_geometric.data import Batch
import torch
import time
import torch.nn as nn
from torch_geometric.nn import GATConv, GCNConv, TAGConv, knn
from torch.nn import Linear, Dropout, Softmax, LeakyReLU
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import global_mean_pool
from hashlib import sha1
from sklearn.tree import DecisionTreeClassifier, plot_tree
from natsort import natsorted, ns

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams["font.size"] = 16
from sklearn.compose import ColumnTransformer, make_column_transformer


from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.tree import DecisionTreeClassifier


TRAIN_PROP = 0.8



In [5]:
# Load CSV From File
dataset_root = os.path.join(".", "data", DATASET_NAME)

raw_dataset = os.path.join(dataset_root, "raw")
processed_dataset = os.path.join(dataset_root, "processed")

patient_dataset = os.path.join(raw_dataset, "Patients.csv")


patient_data = pd.read_csv(patient_dataset, index_col=0)

patient_data

,RESPONSE,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
PATIENTS,,,,,,,,,,,,,,,,,,,,,
C1.43.MO3,1,3.21886,2.92638,0.0,-0.00170,-0.11964,0.0,0.0,1.21708,-0.01241,...,0.48440,0.67825,1.27458,2.56664,3.65988,0.0,2.08857,8.15220,4.08362,2.54912
C1.80.BL,0,2.79139,2.17104,0.0,-0.00170,0.00418,0.0,0.0,0.00721,-0.01241,...,1.77028,0.97612,-0.23791,0.17924,4.36598,0.0,2.48187,9.74258,5.85674,2.97492
C1.76.BL,0,2.11091,1.72769,0.0,-0.00170,-0.11964,0.0,0.0,0.00721,-0.01241,...,1.49443,0.98745,-0.23791,0.78520,5.42406,0.0,3.19363,9.97302,6.49395,3.10468
C1.119.BL,0,0.59019,2.54274,0.0,-0.00170,-0.11964,0.0,0.0,0.00721,-0.01241,...,2.19427,0.11299,-0.11873,1.71442,5.60907,0.0,3.42543,9.48793,6.73700,2.20381
C1.52.BL,0,1.73618,1.99748,0.0,-0.00170,-0.11964,0.0,0.0,0.00721,1.47855,...,1.66573,2.66391,0.91689,2.72109,5.97159,0.0,3.02657,9.51062,6.66191,3.56577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2.62.BL,0,2.32722,2.78940,0.0,0.00308,0.21916,0.0,0.0,-0.00846,0.02251,...,0.88491,1.26625,0.30662,1.58260,5.03738,0.0,2.84432,10.15322,5.90220,1.81392
C2.46.MO3,1,1.84743,2.86524,0.0,0.00308,0.21916,0.0,0.0,-0.00846,0.02251,...,1.54364,1.73128,0.49104,2.50169,5.32422,0.0,3.49868,10.21646,6.20677,2.96830
C2.63.MO3,1,1.30736,2.92271,0.0,0.00308,0.21916,0.0,0.0,-0.00846,0.02251,...,2.26259,1.89086,0.78755,2.93408,4.96799,0.0,3.49305,8.51598,5.97901,3.34918


In [6]:
NUM_EXAMPLES = len(patient_data.index)
NUM_TRAIN = int(TRAIN_PROP * NUM_EXAMPLES)
NUM_TEST = NUM_EXAMPLES - NUM_TRAIN
NUM_VAL = math.ceil(NUM_TRAIN * (1 - TRAIN_PROP))

NUM_VAL

21

In [7]:
r = random.Random(seed_value)

PERSON_ORDER = [i for i in range(NUM_EXAMPLES)]
r.shuffle(PERSON_ORDER)


In [8]:
MASTER_GENE_SET = set([g for g in patient_data][1:])

In [9]:
# Process Graphs.


class Graph:
    def __init__(self, name, nodes, edges, genes, mask):
        self.name = name # a string to identify this graph
        self.nodes = nodes # A dict of node names to the genes in that node.
    
        self.edges = edges # a 2 x num_edges array, where the first row is source, and the second is dest.


        self.genes = genes # a sorted list of genes included in both this graph and the provided patient csv
    
        self.mask = mask # a num_nodes * num_genes boolean array. If mask[i][j] is True, that means gene j is included in node i.

    def edge_matrix(self):
        num_edges = len(self.edges[0])

        matrix = numpy.full((2, 2), False)

        for i in range(num_edges):
            src = self.edges[0][i]
            dst = self.edges[1][i]

            matrix[dst][src] = True

        return matrix


# returns a 2 x num_edges array.
def get_edges(graph_file):
    num_edges = int(graph_file.readline())


    cur_edges = []

    for _ in range(num_edges):
        line = graph_file.readline()

        parts = line.split(",")

        cur_edges.append((str(parts[0]).strip(), str(parts[1]).strip()))

    src, dst = zip(*cur_edges)

    src, dst = list(src), list(dst)

    cur_edges = [src, dst]

    return cur_edges


# cur_nodes is a dict, genes is a sorted list.
def get_nodes_and_genes(graph_file):
    num_nodes = int(graph_file.readline())

    cur_nodes = dict()

    genes = set()
    # process the genes, get the gene map, and the list of genes.
    for _ in range(num_nodes):
        line = graph_file.readline().strip()
        parts = line.split(",")

        cur_genes = parts[1:]

        cur_genes = [g for g in cur_genes if g in MASTER_GENE_SET]

        if ("" not in cur_genes):
            cur_nodes[parts[0]] = cur_genes
    
            genes.update(cur_genes)

    genes = list(genes)
    genes = natsorted(genes)



    return cur_nodes, genes


# a num_nodes x num_genes array. If mask[i][j] is true, that means node i contains gene j.
def get_mask(cur_nodes, genes):
    genes = list(genes)
    genes = natsorted(genes)

    node_list = [n for n in cur_nodes.keys()]
    node_list =  natsorted(node_list)


    mask = []

    for node in node_list:
        cur_genes = cur_nodes[node]

        mask.append([True if g in cur_genes else False for g in genes])


    mask = np.array(mask)

    return mask

In [10]:
graph_dataset = os.path.join(raw_dataset, "Graphs")

graph_files = os.listdir(graph_dataset)
graph_files = natsorted(graph_files)

graph_list = []


# will have as many 0's as there are nodes in the first graph, as many 1's as nodes in the second, etc.

# essentially, lasso_groups[i] tells you which graph the [i'th] col of the dataframe we're making belongs to.

for i, gl in enumerate(graph_files):
    graph_file = os.path.join(graph_dataset, gl)

    graph_file = open(graph_file)

    cur_edges = get_edges(graph_file)

    cur_nodes, genes = get_nodes_and_genes(graph_file)

    mask = get_mask(cur_nodes, genes)

    
    graph_list.append(Graph(gl, cur_nodes, cur_edges, genes, mask))


In [11]:
# making the giant dataframe for group_lasso

lasso_data = np.array(patient_data['RESPONSE'])

lasso_data = np.expand_dims(lasso_data, axis=1)

col_names = ["label"]
lasso_groups = []

for i, graph in enumerate(graph_list):
    genes = graph.genes
    name = graph.name
    lasso_groups.extend([i] * len(genes))

    col_names.extend([f"{name}_{gene}" for gene in genes])
    lasso_data = np.hstack((lasso_data, np.array(patient_data[graph.genes])))

In [12]:
df = pd.DataFrame(lasso_data, patient_data.index, col_names) 

df

,label,AGE-RAGE signaling pathway in diabetic complications.txt_AGER,AGE-RAGE signaling pathway in diabetic complications.txt_AGT,AGE-RAGE signaling pathway in diabetic complications.txt_AGTR1,AGE-RAGE signaling pathway in diabetic complications.txt_AKT3,AGE-RAGE signaling pathway in diabetic complications.txt_BAX,AGE-RAGE signaling pathway in diabetic complications.txt_BCL2,AGE-RAGE signaling pathway in diabetic complications.txt_CASP3,AGE-RAGE signaling pathway in diabetic complications.txt_CCL2,AGE-RAGE signaling pathway in diabetic complications.txt_CCND1,...,p53 signaling pathway.txt_STEAP3,p53 signaling pathway.txt_TEP1,p53 signaling pathway.txt_THBS1,p53 signaling pathway.txt_TNFRSF10B,p53 signaling pathway.txt_TP53,p53 signaling pathway.txt_TP53AIP1,p53 signaling pathway.txt_TP73,p53 signaling pathway.txt_TSC2,p53 signaling pathway.txt_ZMAT3,p53 signaling pathway.txt_ZNF385A
PATIENTS,,,,,,,,,,,,,,,,,,,,,
C1.43.MO3,1.0,5.99755,0.0,0.0,2.73219,6.74621,3.70124,5.07138,-0.00871,-0.00447,...,1.09594,5.31689,4.76415,6.19625,4.43874,0.0,0.0,5.05085,4.23904,7.52960
C1.80.BL,0.0,6.49587,0.0,0.0,3.03913,7.11058,4.91876,3.92118,-0.00871,0.18338,...,1.88375,5.81153,3.20795,4.25891,5.64460,0.0,0.0,6.32223,1.84994,7.81712
C1.76.BL,0.0,6.31102,0.0,0.0,3.30029,6.57437,3.58347,4.69987,-0.00871,-0.00447,...,1.04625,5.41517,2.52663,4.73852,4.35242,0.0,0.0,5.90557,4.19721,6.72206
C1.119.BL,0.0,5.95126,0.0,0.0,3.22844,6.45857,4.47390,4.94380,-0.00871,-0.00447,...,0.30425,6.23749,3.76528,5.75005,4.82226,0.0,0.0,6.17386,4.04938,7.00874
C1.52.BL,0.0,6.34335,0.0,0.0,4.05056,6.60905,5.40336,5.05897,-0.00871,0.27064,...,0.30425,6.47154,3.88717,6.37545,5.41376,0.0,0.0,6.43551,4.15562,7.53452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C2.62.BL,0.0,6.78093,0.0,0.0,3.55648,6.63510,4.65309,4.51811,0.01006,0.00563,...,2.62680,5.51628,3.91596,5.53173,5.57223,0.0,0.0,6.30619,3.30908,7.52000
C2.46.MO3,1.0,6.48353,0.0,0.0,3.80158,6.72330,5.28525,4.47267,0.01006,0.00563,...,2.05364,5.99409,3.18667,6.09746,6.05054,0.0,0.0,6.52211,3.92314,8.06242
C2.63.MO3,1.0,5.82544,0.0,0.0,4.02588,6.71172,5.67849,5.44270,1.30525,0.00563,...,1.39890,5.96492,4.30707,5.36377,5.70351,0.0,0.0,5.79687,4.53810,7.12858


In [13]:
shuffled_df = df.iloc[PERSON_ORDER]


train_df = shuffled_df[0:NUM_TRAIN]
test_df = shuffled_df[NUM_TRAIN:]

train_df

,label,AGE-RAGE signaling pathway in diabetic complications.txt_AGER,AGE-RAGE signaling pathway in diabetic complications.txt_AGT,AGE-RAGE signaling pathway in diabetic complications.txt_AGTR1,AGE-RAGE signaling pathway in diabetic complications.txt_AKT3,AGE-RAGE signaling pathway in diabetic complications.txt_BAX,AGE-RAGE signaling pathway in diabetic complications.txt_BCL2,AGE-RAGE signaling pathway in diabetic complications.txt_CASP3,AGE-RAGE signaling pathway in diabetic complications.txt_CCL2,AGE-RAGE signaling pathway in diabetic complications.txt_CCND1,...,p53 signaling pathway.txt_STEAP3,p53 signaling pathway.txt_TEP1,p53 signaling pathway.txt_THBS1,p53 signaling pathway.txt_TNFRSF10B,p53 signaling pathway.txt_TP53,p53 signaling pathway.txt_TP53AIP1,p53 signaling pathway.txt_TP73,p53 signaling pathway.txt_TSC2,p53 signaling pathway.txt_ZMAT3,p53 signaling pathway.txt_ZNF385A
PATIENTS,,,,,,,,,,,,,,,,,,,,,
C2.127.BL,0.0,4.14177,0.0,0.0,2.81147,4.61906,3.76450,4.29467,0.01006,0.00563,...,0.82010,5.27701,2.60440,4.73189,3.73687,0.0,0.0,4.51334,3.02886,4.80906
C2.171.BL,0.0,6.21947,0.0,0.0,2.46761,6.01984,4.00206,3.05361,0.01006,0.00563,...,0.04900,4.52441,4.89572,4.14827,4.77172,0.0,0.0,5.52115,3.53559,6.27621
C1.71.BL,0.0,5.65233,0.0,0.0,5.06637,4.68703,6.05423,6.06115,-0.00871,-0.00447,...,1.14295,6.46958,4.98978,6.81019,4.98847,0.0,0.0,5.69282,5.02169,5.54980
C1.68.MO3,1.0,6.14886,0.0,0.0,4.64227,6.48077,5.71867,5.03559,-0.00871,-0.00447,...,2.65425,7.07311,4.41224,6.31587,6.45670,0.0,0.0,6.71887,4.53389,7.83055
C1.49.MO3,1.0,6.15527,0.0,0.0,5.17006,5.94297,6.14394,6.45175,-0.00871,0.83320,...,2.99977,6.69760,2.99441,6.89006,4.90126,0.0,0.0,5.43777,3.91167,7.13889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C1.6.MO3,1.0,5.79895,0.0,0.0,4.75792,5.86039,5.22564,5.62432,-0.00871,-0.00447,...,1.68640,6.69558,3.58143,6.67969,5.15118,0.0,0.0,6.49725,4.64681,6.20724
C1.23.BL,0.0,6.42759,0.0,0.0,2.57025,5.32276,3.41369,4.08260,-0.00871,-0.00447,...,0.33660,5.86978,5.12757,5.28727,4.10547,0.0,0.0,5.53819,3.36857,6.14361
C1.99.MO3,1.0,7.11203,0.0,0.0,3.17040,6.95960,4.05756,3.56470,-0.00871,-0.00447,...,0.30425,4.74403,3.05448,5.07662,5.36194,0.0,0.0,6.31332,2.53539,7.92165


# Preparing For GNN Explain

In [14]:
len(graph_list[0].genes)

54

In [15]:
len(graph_list[0].genes)

54

In [16]:
class Graph:
    def __init__(self, name, nodes, edges, genes, mask):
        self.name = name # a string to identify this graph
        self.nodes = nodes # A dict of node names to the genes in that node.
    
        self.edges = edges # a 2 x num_edges array, where the first row is source, and the second is dest.


        self.genes = genes # a sorted list of genes included in both this graph and the provided patient csv
    
        self.mask = mask # a num_nodes * num_genes boolean array. If mask[i][j] is True, that means gene j is included in node i.

    def edge_matrix(self):
        num_edges = len(self.edges[0])

        matrix = numpy.full((2, 2), False)

        for i in range(num_edges):
            src = self.edges[0][i]
            dst = self.edges[1][i]

            matrix[dst][src] = True

        return matrix

In [17]:
import os
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import lmdb
import pickle
from torch_geometric.data import Batch
import torch
import time
import torch.nn as nn
from torch_geometric.nn import GATConv, GCNConv, TAGConv, knn
from torch.nn import Linear, Dropout, Softmax, LeakyReLU
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import global_mean_pool
import pandas as pd

from torch_geometric.data import Data




# processing Graph 


master_edge_index = torch.tensor([[],[]])

nodes_before = 0
genes_before = 0

nodes_per_graph = []
edges_per_graph = []
genes_per_graph = []

num_to_name_dicts = []
# create a giant graph
for i, graph in enumerate(graph_list):
    # change this to add as well.
    cur_example = graph_list[i]
    cur_nodes = [k for k in cur_example.nodes.keys()]
    
    cur_nodes = natsorted(cur_nodes)

    node_int_dict = {n:i for i, n in enumerate(cur_nodes)}
    int_node_dict = {i:n for i, n in enumerate(cur_nodes)}
    num_to_name_dicts.append(int_node_dict)
    
    num_nodes = len(cur_nodes)
    adjacency = torch.diag(torch.tensor([1] * num_nodes))
    post_message_vectors = []
    num_nodes = adjacency.shape[0]
    num_genes = len(cur_example.genes)

    cur_edges = np.array(cur_example.edges)
    cur_edges[0] = np.array([node_int_dict[e] for e in cur_edges[0]])
    cur_edges[1] = np.array([node_int_dict[e] for e in cur_edges[1]])

    cur_edges = cur_edges.astype(int)

    cur_edges  = torch.tensor(cur_edges)

    cur_num_nodes = len(cur_example.nodes)

    master_edge_index = torch.hstack((master_edge_index, cur_edges + nodes_before))


    nodes_before += cur_num_nodes
    genes_before += len(cur_example.genes)

    nodes_per_graph.append(cur_num_nodes)
    genes_per_graph.append(len(cur_example.genes))
    edges_per_graph.append(len(cur_edges[0]))


num_genes = genes_before
num_nodes = nodes_before

(num_genes,shuffled_df.shape[1], num_nodes)

(11167, 11168, 13876)

In [18]:
num_to_name_dicts

[{0: '8',
  1: '9',
  2: '16',
  3: '20',
  4: '22',
  5: '23',
  6: '26',
  7: '31',
  8: '32',
  9: '34',
  10: '38',
  11: '39',
  12: '40',
  13: '43',
  14: '44',
  15: '46',
  16: '47',
  17: '48',
  18: '49',
  19: '50',
  20: '51',
  21: '52',
  22: '53',
  23: '54',
  24: '56',
  25: '59',
  26: '60',
  27: '64',
  28: '69',
  29: '74',
  30: '75',
  31: '78',
  32: '79',
  33: '85',
  34: '86',
  35: '88',
  36: '92',
  37: '94',
  38: '96',
  39: '101',
  40: '102',
  41: '103',
  42: '104',
  43: '105',
  44: '110',
  45: '111',
  46: '113',
  47: '124',
  48: '127',
  49: '130',
  50: '133',
  51: '147',
  52: '150',
  53: '167',
  54: '168',
  55: '175',
  56: '177',
  57: '191'},
 {0: '8',
  1: '9',
  2: '11',
  3: '12',
  4: '13',
  5: '14',
  6: '15',
  7: '30',
  8: '31',
  9: '34',
  10: '44',
  11: '49',
  12: '50',
  13: '52',
  14: '53',
  15: '57',
  16: '67',
  17: '87',
  18: '92',
  19: '99',
  20: '103',
  21: '105',
  22: '109',
  23: '110',
  24: '112',
  2

In [19]:
# stores a giant sparse matrix for each patient. There are as many collumns as total number of genes

# and as many rows as number of nodes across all graphs.



giant_path = os.path.join(processed_dataset, "Giant_Matrix_Input")

giant_env = lmdb.open(giant_path, map_size=6 * 10**9)  # Adjust map_size as needed

with giant_env.begin(write=True) as txn:
    for row_number, (_, row) in enumerate(shuffled_df.iterrows()):
        print(row_number)
        target = row[0]
        cur_row = torch.tensor(np.array(row)[1:])
        cur_matrix = torch.zeros((num_nodes, num_genes))
        prev_genes = 0
        prev_nodes = 0

        for graph in graph_list:
            cur_num_nodes = (len(graph.nodes))
            cur_num_genes = (len(graph.genes))

            cur_data = cur_row[prev_genes:cur_num_genes + prev_genes]

            cur_data = cur_data.repeat(cur_num_nodes, 1)
            cur_mask = torch.tensor(graph.mask)
            cur_data = cur_data.masked_fill(~cur_mask, 0)


            cur_matrix[prev_nodes:prev_nodes + cur_num_nodes, prev_genes: prev_genes + cur_num_genes] = cur_data

            prev_genes += cur_num_genes
            prev_nodes += cur_num_nodes

        print(cur_matrix.shape)
        txn.put(str(row_number).encode(), pickle.dumps((cur_matrix.to_sparse(), target)))
    
    txn.put("Graph".encode(), pickle.dumps(master_edge_index))
    txn.put("num_genes".encode(), pickle.dumps(num_genes))
    txn.put("num_nodes".encode(), pickle.dumps(num_nodes))
    txn.put("nodes_per_graph".encode(), pickle.dumps(nodes_per_graph))
    txn.put("edges_per_graph".encode(), pickle.dumps(edges_per_graph))
    txn.put("genes_per_graph".encode(), pickle.dumps(genes_per_graph))
    txn.put("num_to_name_dicts".encode(), pickle.dumps(num_to_name_dicts))

0
torch.Size([13876, 11167])


/tmp/ipykernel_304441/996619229.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = row[0]


1
torch.Size([13876, 11167])
2
torch.Size([13876, 11167])
3
torch.Size([13876, 11167])
4
torch.Size([13876, 11167])
5
torch.Size([13876, 11167])
6
torch.Size([13876, 11167])
7
torch.Size([13876, 11167])
8
torch.Size([13876, 11167])
9
torch.Size([13876, 11167])
10
torch.Size([13876, 11167])
11
torch.Size([13876, 11167])
12
torch.Size([13876, 11167])
13
torch.Size([13876, 11167])
14
torch.Size([13876, 11167])
15
torch.Size([13876, 11167])
16
torch.Size([13876, 11167])
17
torch.Size([13876, 11167])
18
torch.Size([13876, 11167])
19
torch.Size([13876, 11167])
20
torch.Size([13876, 11167])
21
torch.Size([13876, 11167])
22
torch.Size([13876, 11167])
23
torch.Size([13876, 11167])
24
torch.Size([13876, 11167])
25
torch.Size([13876, 11167])
26
torch.Size([13876, 11167])
27
torch.Size([13876, 11167])
28
torch.Size([13876, 11167])
29
torch.Size([13876, 11167])
30
torch.Size([13876, 11167])
31
torch.Size([13876, 11167])
32
torch.Size([13876, 11167])
33
torch.Size([13876, 11167])
34
torch.Size([1387

In [ ]:
import os
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import lmdb
import pickle
from torch_geometric.data import Batch
import torch
import time
import torch.nn as nn
from torch_geometric.nn import GATConv, GCNConv, TAGConv, knn
from torch.nn import Linear, Dropout, Softmax, LeakyReLU
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import global_mean_pool
import pandas as pd

from torch_geometric.utils import to_dense_adj, add_self_loops


lmdb_path = os.path.join(processed_dataset, "graph_first_lmdb")
os.makedirs(os.path.dirname(lmdb_path), exist_ok=True)


env = lmdb.open(lmdb_path, map_size=6 * 10**9)  # Adjust map_size as needed

# tracks the number of genes before the current graph

# starts at 1 for the label
prev_genes = 1

with env.begin(write=True) as txn:

    for graph_num, graph in enumerate(graph_list):
        cur_nodes = [k for k in graph.nodes.keys()]
        
        cur_nodes = natsorted(cur_nodes)

        node_int_dict = {n:i for i, n in enumerate(cur_nodes)}
        
        num_nodes = len(cur_nodes)
        adjacency = torch.diag(torch.tensor([1] * num_nodes))
        post_message_vectors = []
        num_nodes = adjacency.shape[0]
        num_genes = len(graph.genes)
        graph_name = "graph_{}".format(graph_num)

        cur_edges = np.array(graph.edges)
        cur_edges[0] = np.array([node_int_dict[e] for e in cur_edges[0]])
        cur_edges[1] = np.array([node_int_dict[e] for e in cur_edges[1]])

        cur_edges = cur_edges.astype(int)

        # complete adjacency matrix
        for edge_num in range(len(cur_edges[1])):
            source = cur_edges[0][edge_num]
            dest = cur_edges[1][edge_num]
            adjacency[dest][source] = 1

        edge_index_with_loops, _ = add_self_loops(torch.tensor(cur_edges))

        pyg_adj = to_dense_adj(edge_index_with_loops)[0]

        adjacency = adjacency.float()



        # I'm repeat interleaving wrong.
        interleaving_mask = torch.tensor(graph.mask).repeat(NUM_EXAMPLES, 1)
        interleaving_data = torch.repeat_interleave(torch.tensor(np.array(shuffled_df.iloc[:,prev_genes:num_genes + prev_genes])), num_nodes, dim=0)
        labels = torch.repeat_interleave(torch.tensor(shuffled_df.label).unsqueeze(dim=1),num_nodes, dim=0)

        # fix natural sort error
        
        #interleaving_mask[:, [26, 27]] = interleaving_mask[:, [27, 26]]
        #interleaving_data[:, [26, 27]] = interleaving_data[:, [27, 26]]
        
        cur_data = interleaving_data.masked_fill(~interleaving_mask, 0)

        key_index = 0
        
        
        # Rearrange, so that the various different patients are stacked horizontally, instead of vertically.
        chunks = torch.chunk(cur_data, NUM_EXAMPLES, dim=0)
        cur_data = torch.cat(chunks, dim=1)
        
        
        cur_data = torch.matmul(adjacency.double(), cur_data)
        
        
        chunks = torch.chunk(cur_data, NUM_EXAMPLES, dim=1)
        cur_data = torch.cat(chunks, dim=0) 
        cur_data = torch.hstack((cur_data, labels))

        """
        print(torch.max(torch.abs(cur_data[:,-1] - correct[:,-1])))
        print(torch.sum(cur_data[:,-1] - correct[:,-1]))
        print(cur_data[:,-1])
        print(correct[:,-1])
        print(cur_data.shape)
        1/0"""

            

        # store the entire graph into the database as a dataframe, with (128 * nodes) rows and num_genes + 1 (for target) col
        col_names = list(shuffled_df.columns[prev_genes:prev_genes+num_genes])
        col_names.append("target")
        graph_df = pd.DataFrame(cur_data, columns=col_names)
        txn.put(graph_name.encode(), pickle.dumps(graph_df))
        prev_genes += num_genes

    txn.put("num_graphs".encode(), pickle.dumps(len(graph_list)))

In [ ]:
def get_item(idx):
    with env.begin() as txn:
        item_bytes = txn.get("graph_{}".format(idx).encode())

        return pickle.loads(item_bytes)


def get_num_graphs():
    with env.begin() as txn:
        item_bytes = txn.get("num_graphs".encode())

        return pickle.loads(item_bytes)

In [ ]:
#[g[1] for g in graphs[2:]] == [g[1] for g in graphs_two]

In [ ]:
def get_graph_info(idx):
    df = get_item(idx)
    target_col = 'target'

    y = df[target_col]
    y = y.astype(int)
    
    X = df.copy()
    X = X.drop(target_col, axis=1)

    return X, y


In [ ]:
get_num_graphs()

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder,MaxAbsScaler
from sklearn.impute import SimpleImputer

def divide_and_scale(X, y):
    num_nodes = int(X.shape[0] / NUM_EXAMPLES)
    X_train = X[0:NUM_TRAIN * num_nodes]
    y_train = y[0:NUM_TRAIN * num_nodes]
    
    X_test = X[NUM_TRAIN * num_nodes:]
    y_test = y[NUM_TRAIN * num_nodes:]
    X_train.shape
    
    transformer = MaxAbsScaler()
    transformer.fit(X_train)
    
    X_train = pd.DataFrame(transformer.transform(X_train))
    X_test = pd.DataFrame(transformer.transform(X_test))
    return X_train, X_test, y_train, y_test, num_nodes, transformer

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

def get_acc(model, X, y, predict_type):
    y_pred = model.predict(X)
    acc = accuracy_score(y, y_pred)
    
    return acc

def get_auc(model, X, y, predict_type):
    y_logits = model.decision_function(X)
    auc = roc_auc_score(y, y_logits)
    return auc

In [ ]:
def cross_validation(X_train, y_train, i, num_nodes):
    num_val_nodes = ( NUM_VAL * num_nodes)
    if(i == 4):
        X_val = X_train[- num_val_nodes:]
        y_val = y_train[- num_val_nodes:]
        
        X_train = X_train[0:-num_val_nodes]
        y_train = y_train[0:-num_val_nodes]
    else:
        X_val = X_train[i * num_val_nodes:(i + 1) * num_val_nodes]
        y_val = y_train[i * num_val_nodes:(i + 1) * num_val_nodes]
        
        X_train = np.concatenate((X_train[:i * num_val_nodes], X_train[(i + 1) * num_val_nodes:]))
        y_train = np.concatenate((y_train[:i * num_val_nodes], y_train[(i + 1) * num_val_nodes:]))


    return X_train, y_train, X_val, y_val

In [ ]:
final_train_aucs = []
final_val_aucs = []

accuracy_scores = []
print("Hi!")
for k in range(5):
    print("K: {}".format(k))
    X_train_patient = []
    X_val_patient = []
    
    # 0 is node level train, 1 is node level val, 2 is graph level train, 3 is graph level val.
    accuracies = [[], [], [], []]
    for graph_num in range(get_num_graphs()):
        if graph_num % 50 == 0:
            print(graph_num)
        cur_X, cur_Y = get_graph_info(graph_num)
        
        X_train, X_test, y_train, y_test, num_nodes, _ = divide_and_scale(cur_X, cur_Y)
        
        X_train, y_train, X_val, y_val =  cross_validation(X_train, y_train, k, num_nodes)
        
        model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
        model.fit(X_train, y_train)

        
        acc_train = get_auc(model, X_train, y_train, "Train")
        acc_val = get_auc(model, X_val, y_val, "Validation")
    
        accuracies[0].append(acc_train)
        accuracies[1].append(acc_val)
    
        logits_train = model.decision_function(X_train).reshape(NUM_TRAIN - NUM_VAL, -1)
        logits_val = model.decision_function(X_val).reshape(NUM_VAL, -1)
    
        scaler = StandardScaler()
    
        scaler.fit(logits_train)
    
        logits_train = scaler.transform(logits_train)
        logits_val = scaler.transform(logits_val)
        
        y_graph_train = y_train[::logits_train.shape[1]]
        y_graph_val = y_val[::logits_train.shape[1]]
        
        graph_level_model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
        
        graph_level_model.fit(logits_train, y_graph_train)
        
        acc_train_graph = get_auc(graph_level_model, logits_train, y_graph_train, "Graph-Level Train")
        acc_val_graph = get_auc(graph_level_model, logits_val, y_graph_val, "Graph-Level Validation")
        
        
        accuracies[2].append(acc_train_graph)
        accuracies[3].append(acc_val_graph)
    
        
        X_train_patient.append(graph_level_model.decision_function(logits_train))
        X_val_patient.append(graph_level_model.decision_function(logits_val))
    
    X_train_patient = np.array(X_train_patient).T
    X_val_patient = np.array(X_val_patient).T
    
    scaler.fit(X_train_patient)
    X_train_patient = scaler.transform(X_train_patient)
    X_val_patient = scaler.transform(X_val_patient)
    
    patient_level_model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
    
    patient_level_model.fit(X_train_patient, y_graph_train)
    
    final_train_auc = get_auc(patient_level_model, X_train_patient, y_graph_train, "Patient Level Train")
    final_val_auc = get_auc(patient_level_model, X_val_patient, y_graph_val, "Patient Level Val")
    
    print("Final Train AUC: {}".format(final_train_auc))
    print("Final Val AUC: {}".format(final_val_auc))
    
    final_train_aucs.append(final_train_auc)
    final_val_aucs.append(final_val_auc)
    accuracy_scores.append(accuracies)

In [ ]:
pd.concat([y_graph_train, y_graph_val]).value_counts()

In [ ]:
final_train_aucs

In [ ]:
fig, axes = plt.subplots(5, 4, figsize=(16, 20), sharey=True)

# Define labels
row_labels = [f'K fold, k = {i}' for i in range(5)]
col_labels = [
    'Node-level Train Accuracy',
    'Node-level Val Accuracy',
    'Graph-level Train Accuracy',
    'Graph-level Val Accuracy'
]

# Plot boxplots
for i in range(5):
    for j in range(4):
        axes[i, j].boxplot(accuracy_scores[i][j])
        axes[i, j].grid(True)
        
        # Label only the top row columns
        if i == 0:
            axes[i, j].set_title(col_labels[j], fontsize=12, pad=10)
        
        # Label only the first column rows
        if j == 0:
            axes[i, j].set_ylabel(row_labels[i], fontsize=12)

plt.tight_layout(rect=[0, 0, 1, 0.97])  # leave space for top labels
fig.suptitle('Accuracy Boxplots Across K Folds', fontsize=16)
plt.show()


In [ ]:
final_val_aucs

In [ ]:
def k_fold_run(C_val, l1_ratio):
    print("(C, l1): ({}, {})".format(C_val, l1_ratio))
    final_train_aucs = []
    final_val_aucs = []
    
    accuracy_scores = []
    print("Hi!")
    for k in range(5):
        print("K: {}".format(k))
        X_train_patient = []
        X_val_patient = []
        
        # 0 is node level train, 1 is node level val, 2 is graph level train, 3 is graph level val.
        accuracies = [[], [], [], []]
        for graph_num in range(get_num_graphs()):
            if graph_num % 50 == 0:
                print(graph_num)
            cur_X, cur_Y = get_graph_info(graph_num)
            
            X_train, X_test, y_train, y_test, num_nodes, _ = divide_and_scale(cur_X, cur_Y)
            
            X_train, y_train, X_val, y_val =  cross_validation(X_train, y_train, k, num_nodes)
            
            model = LogisticRegression(penalty="elasticnet", C=C_val, solver='saga', l1_ratio=l1_ratio, max_iter=10000)
            model.fit(X_train, y_train)
            
            acc_train = get_auc(model, X_train, y_train, "Train")
            acc_val = get_auc(model, X_val, y_val, "Validation")
        
            accuracies[0].append(acc_train)
            accuracies[1].append(acc_val)
        
            logits_train = model.decision_function(X_train).reshape(NUM_TRAIN - NUM_VAL, -1)
            logits_val = model.decision_function(X_val).reshape(NUM_VAL, -1)
        
            scaler = StandardScaler()
        
            scaler.fit(logits_train)
        
            logits_train = scaler.transform(logits_train)
            logits_val = scaler.transform(logits_val)
            
            y_graph_train = y_train[::logits_train.shape[1]]
            y_graph_val = y_val[::logits_train.shape[1]]
            
            graph_level_model = LogisticRegression(penalty="elasticnet", C=C_val, solver='saga', l1_ratio=l1_ratio, max_iter=10000)
            
            graph_level_model.fit(logits_train, y_graph_train)
            
            acc_train_graph = get_auc(graph_level_model, logits_train, y_graph_train, "Graph-Level Train")
            acc_val_graph = get_auc(graph_level_model, logits_val, y_graph_val, "Graph-Level Validation")
            
            
            accuracies[2].append(acc_train_graph)
            accuracies[3].append(acc_val_graph)
        
            
            X_train_patient.append(graph_level_model.decision_function(logits_train))
            X_val_patient.append(graph_level_model.decision_function(logits_val))
        
        X_train_patient = np.array(X_train_patient).T
        X_val_patient = np.array(X_val_patient).T
        
        scaler.fit(X_train_patient)
        X_train_patient = scaler.transform(X_train_patient)
        X_val_patient = scaler.transform(X_val_patient)
        
        patient_level_model = LogisticRegression(penalty="elasticnet", C=C_val, solver='saga', l1_ratio=l1_ratio, max_iter=10000)
        
        patient_level_model.fit(X_train_patient, y_graph_train)
        
        final_train_auc = get_auc(patient_level_model, X_train_patient, y_graph_train, "Patient Level Train")
        final_val_auc = get_auc(patient_level_model, X_val_patient, y_graph_val, "Patient Level Val")
        
        print("Final Train AUC: {}".format(final_train_auc))
        print("Final Val AUC: {}".format(final_val_auc))
        
        final_train_aucs.append(final_train_auc)
        final_val_aucs.append(final_val_auc)
        accuracy_scores.append(accuracies)
    
    average_train_auc = sum(final_train_aucs) / len(final_train_aucs)
    avg_val_auc = sum(final_val_aucs) / len(final_val_aucs)

    key = (C_val, l1_ratio)

    return average_train_auc, avg_val_auc

In [ ]:
C_vals = [0.05 * i for i in range(1, 8)]
l1_ratios = [0.2 * i for i in range(1, 6)]

train_aucs = []
val_aucs = []


for l1_ratio in l1_ratios:
    cur_train_auc = []
    cur_val_auc = []
    
    for C_val in C_vals:
        train_auc, val_auc = k_fold_run(C_val, l1_ratio)
        cur_train_auc.append(train_auc)
        cur_val_auc.append(val_auc)

    train_aucs.append(cur_train_auc)
    val_aucs.append(cur_val_auc)

In [ ]:
def make_heatmap(title, data, x_ticks, y_ticks, xlabel = "L1-Reg", y_label = "Group Reg"):
    fig, ax = plt.subplots()
    im = ax.imshow(data, cmap="cividis")
    
    # Show all ticks and label them with the respective list entries
    ax.set_xticks(range(len(x_ticks)), labels=[f"{gl:.2f}" for gl in x_ticks],
                  rotation=45, ha="right", rotation_mode="anchor")
    ax.set_yticks(range(len(y_ticks)), labels=[f"{l1:.2f}" for l1 in y_ticks])
    
    text_size = min(10, 300 / max(len(x_ticks), len(y_ticks)))

    
    train_auc = np.round(data, decimals=3)
    # Loop over data dimensions and create text annotations.
    for i in range(len(y_ticks)):
        for j in range(len(x_ticks)):
            text = ax.text(j, i, f"{train_auc[i, j]}",
                           ha="center", va="center", color="red", fontsize=text_size)
    
    ax.set_title(title)

    plt.xlabel(xlabel)
    plt.ylabel(y_label)
    plt.show()



In [ ]:
np.array(train_aucs[::-1])

In [ ]:
make_heatmap("Train_AUC", train_aucs[::-1], C_vals, l1_ratios[::-1], "C Vals", "L1-Ratio") 

In [ ]:
make_heatmap("Val AUC", val_aucs[::-1], C_vals, l1_ratios[::-1], "C Vals", "L1-Ratio") 

In [ ]:
max_index_flat = np.argmax(val_aucs)
val_aucs = np.array(val_aucs)

# Convert the flat index to a coordinate tuple
best_coords = np.unravel_index(max_index_flat, val_aucs.shape)
print(f"Coordinates of max value: {best_coords}") # Output: (1, 0)
print(f"Maximum value: {val_aucs[best_coords]}") # Output: 60

In [ ]:
best_Cval = C_vals[best_coords[0]]
best_ratio = l1_ratios[best_coords[1]]

print(f"Best C-val and L1-Ratio: {(best_Cval, best_ratio)}")

In [ ]:
C_val = best_Cval
l1_ratio = best_ratio

X_train_patient = []
X_test_patient = []

# 0 is node level train, 1 is node level test, 2 is graph level train, 3 is graph level test.
accuracies = [[], [], [], []]

node_models = []
node_transformers = []
node_logits = []

graph_models = []
graph_transformers = []
graph_logits = []

final_logits = []

for graph_num in range(get_num_graphs()):
    if graph_num % 50 == 0:
        print(graph_num)
    cur_X, cur_Y = get_graph_info(graph_num)
    
    X_train, X_test, y_train, y_test, num_nodes, transformer = divide_and_scale(cur_X, cur_Y)

    node_transformers.append(transformer)
        
    model = LogisticRegression(penalty="elasticnet", C=C_val, solver='saga', l1_ratio=l1_ratio, max_iter=100000)
    model.fit(X_train, y_train)
    
    acc_train = get_auc(model, X_train, y_train, "Train")
    acc_test = get_auc(model, X_test, y_test, "Test")

    accuracies[0].append(acc_train)
    accuracies[1].append(acc_test)

    node_models.append(model)

    logits_train = model.decision_function(X_train).reshape(NUM_TRAIN, -1)
    logits_test = model.decision_function(X_test).reshape((NUM_EXAMPLES - NUM_TRAIN), -1)

    scaler = StandardScaler()

    scaler.fit(logits_train)
    graph_transformers.append(scaler)


    logits_train = scaler.transform(logits_train)
    logits_test = scaler.transform(logits_test)
    node_logits.append(np.vstack((logits_train, logits_test)))
    y_graph_train = y_train[::logits_train.shape[1]]
    y_graph_test = y_test[::logits_train.shape[1]]
    
    graph_level_model = LogisticRegression(penalty="elasticnet", C=C_val, solver='saga', l1_ratio=l1_ratio, max_iter=10000)
    
    graph_level_model.fit(logits_train, y_graph_train)
    
    acc_train_graph = get_auc(graph_level_model, logits_train, y_graph_train, "Graph-Level Train")
    acc_test_graph = get_auc(graph_level_model, logits_test, y_graph_test, "Graph-Level Test")
    
    
    accuracies[2].append(acc_train_graph)
    accuracies[3].append(acc_test_graph)

    graph_models.append(graph_level_model)

    
    X_train_patient.append(graph_level_model.decision_function(logits_train))
    X_test_patient.append(graph_level_model.decision_function(logits_test))

X_train_patient = np.array(X_train_patient).T
X_test_patient = np.array(X_test_patient).T

scaler = StandardScaler()
scaler.fit(X_train_patient)
X_train_patient = scaler.transform(X_train_patient)
X_test_patient = scaler.transform(X_test_patient)

patient_level_model = LogisticRegression(penalty="elasticnet", C=C_val, solver='saga', l1_ratio=l1_ratio, max_iter=10000)

patient_level_model.fit(X_train_patient, y_graph_train)

final_graph_train_auc = get_auc(patient_level_model, X_train_patient, y_graph_train, "Patient Level Train")
final_graph_test_auc = get_auc(patient_level_model, X_test_patient, y_graph_test, "Patient Level Val")

print("Final Train AUC: {}".format(final_graph_train_auc))
print("Final Test AUC: {}".format(final_graph_test_auc))



In [ ]:
num_nodes = []
num_genes = []

for graph_num in range(get_num_graphs()):
    cur_X, cur_Y = get_graph_info(graph_num)
    assert(cur_X.shape[0] / 128 % 1 == 0)
    num_nodes.append(int(cur_X.shape[0] / 128))
    num_genes.append(cur_X.shape[1])


print(sum(num_genes))

In [ ]:
final_node_logits = np.hstack(node_logits)
final_graph_logits = np.vstack((X_train_patient, X_test_patient))
final_output_logits = np.hstack((patient_level_model.decision_function(X_train_patient), patient_level_model.decision_function(X_test_patient)))
ground_truth = np.hstack((y_graph_train, y_graph_test)).shape
node_transform_scales = np.hstack([nt.scale_ for nt in node_transformers])
graph_transform_scales = np.hstack([gt.scale_ for gt in graph_transformers])
graph_transform_means = np.hstack([gt.mean_ for gt in graph_transformers])
final_transform_scales = scaler.scale_
final_transform_means = scaler.mean_

In [ ]:
node_intercepts = np.array([nm.intercept_[0] for nm in node_models])
graph_intercepts = np.array([gm.intercept_[0] for gm in graph_models])
final_intercept = patient_level_model.intercept_[0]
final_coefs = patient_level_model.coef_

In [ ]:
node_coefs = np.zeros((get_num_graphs(), sum(num_genes)))

for i in range(get_num_graphs()):
    start = sum(num_genes[0:i])
    end = start + num_genes[i]
    coefs = node_models[i].coef_

    assert(coefs.shape[1] == (end-start))
    node_coefs[i][start:end] = coefs

node_coefs = node_coefs.T

In [ ]:
graph_coefs = np.zeros((get_num_graphs(), sum(num_nodes)))

for i in range(get_num_graphs()):
    start = sum(num_nodes[0:i])
    end = start + num_nodes[i]
    coefs = graph_models[i].coef_

    assert(coefs.shape[1] == (end-start))
    graph_coefs[i][start:end] = coefs

graph_coefs = graph_coefs.T

In [ ]:
node_intercepts = np.array([nm.intercept_[0] for nm in node_models])
graph_intercepts = np.array([gm.intercept_[0] for gm in graph_models])
final_intercept = patient_level_model.intercept_[0]
final_coefs = patient_level_model.coef_

In [ ]:
save_path = os.path.join(processed_dataset, "Saved_Info")

saved_env = lmdb.open(save_path, map_size=6 * 10**9)  # Adjust map_size as needed

with saved_env.begin(write=True) as txn:
    txn.put("final_node_logits".encode(), pickle.dumps(final_node_logits))
    txn.put("final_graph_logits".encode(), pickle.dumps(final_graph_logits))
    txn.put("final_output_logits".encode(), pickle.dumps(final_output_logits))
    
    txn.put("ground_truth".encode(), pickle.dumps(ground_truth))
    
    txn.put("node_transform_scales".encode(), pickle.dumps(node_transform_scales))
    txn.put("graph_transform_scales".encode(), pickle.dumps(graph_transform_scales))
    txn.put("graph_transform_means".encode(), pickle.dumps(graph_transform_means))
    txn.put("final_transform_scales".encode(), pickle.dumps(final_transform_scales))
    txn.put("final_transform_means".encode(), pickle.dumps(final_transform_means))
    
    txn.put("node_intercepts".encode(), pickle.dumps(node_intercepts))
    txn.put("graph_intercepts".encode(), pickle.dumps(graph_intercepts))
    txn.put("final_intercept".encode(), pickle.dumps(final_intercept))
    
    txn.put("final_coefs".encode(), pickle.dumps(final_coefs))
    txn.put("graph_coefs".encode(), pickle.dumps(graph_coefs))
    txn.put("node_coefs".encode(), pickle.dumps(node_coefs))
    
    txn.put("num_nodes".encode(), pickle.dumps(num_nodes))
    txn.put("num_genes".encode(), pickle.dumps(num_genes))


In [ ]:
train_y = train_df['label']

train_X = train_df.copy()

train_X = train_df.drop('label', axis=1)


train_y

In [ ]:
test_y = test_df['label']

test_X = test_df.copy()

test_X = test_df.drop('label', axis=1)

test_X

In [ ]:
groups = lasso_groups

In [ ]:
gl = LogisticGroupLasso(
    groups=groups,
    group_reg=0.00,
    l1_reg=0,
    scale_reg="inverse_group_size",
    subsampling_scheme=1,
    supress_warning=True,
)

gl.fit(train_X, train_y)

In [ ]:
# Extract info from estimator
pred_c = gl.predict(train_X)
sparsity_mask = gl.sparsity_mask_
w_hat = gl.coef_

# Compute performance metrics
accuracy = (pred_c == train_y).mean()

# Print results: We are hoping to overfit at this point.
print(f"Number variables: {len(sparsity_mask)}")
print(f"Number of chosen variables: {sparsity_mask.sum()}")
print(f"Accuracy: {accuracy}")

In [ ]:
# Extract info from estimator
pred_c = gl.predict(train_X)
sparsity_mask = gl.sparsity_mask_
w_hat = gl.coef_

# Compute performance metrics
accuracy = (pred_c == train_y).mean()

# Print results: We are hoping to overfit at this point.
print(f"Number variables: {len(sparsity_mask)}")
print(f"Number of chosen variables: {sparsity_mask.sum()}")
print(f"Accuracy: {accuracy}")

In [ ]:
def cross_validation(X_train, y_train, i):
    if(i == 4):
        X_val = X_train[- NUM_VAL:]
        y_val = y_train[- NUM_VAL:]
        
        X_train = X_train[0:-NUM_VAL]
        y_train = y_train[0:-NUM_VAL]
    else:
        X_val = X_train[i * NUM_VAL:(i + 1) * NUM_VAL]
        y_val = y_train[i * NUM_VAL:(i + 1) * NUM_VAL]
        
        X_train = np.concatenate((X_train[:i * NUM_VAL], X_train[(i + 1) * NUM_VAL:]))
        y_train = np.concatenate((y_train[:i * NUM_VAL], y_train[(i + 1) * NUM_VAL:]))


    return X_train, y_train, X_val, y_val

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

def predict(X, true_y, gl):
    # Extract info from estimator
    pred_y = gl.predict(X)
    sparsity_mask = gl.sparsity_mask_
    w_hat = gl.coef_
    
    # Compute performance metrics
    accuracy = (pred_y == true_y).mean()
    
    # Print results: We are hoping to overfit at this point.
    return accuracy

def predict_auc(X, true_y, gl):
    
    pred_y = gl.predict_proba(X)[:,1]
    auc = roc_auc_score(true_y, pred_y)

    return auc

In [ ]:
original_X_train = train_X
orig_y_train = train_y


X_train, y_train, X_val, y_val = cross_validation(original_X_train, orig_y_train, 0)

gl = LogisticGroupLasso(
    groups=groups,
    group_reg=0.00,
    l1_reg=0,
    scale_reg="inverse_group_size",
    subsampling_scheme=1,
    supress_warning=True,
)

X_train

In [ ]:
import warnings
warnings.filterwarnings('ignore')





train_auc = []

val_auc = []




for gr in group_reg_lesso:
    train_auc_row = []
    val_auc_row = []
    for lreg in l1_reg_lasso:
        print("\nGroup Reg: {:.2f} \nL1 Reg: {:.2f}".format(gr, lreg))
        cur_val_auc = []
        cur_train_auc = []
        for k in range(5):
            print("K-fold K: {}".format(k))
            X_train, y_train, X_val, y_val = cross_validation(original_X_train, orig_y_train, k)

            gl = LogisticGroupLasso(
                groups=groups,
                group_reg=gr,
                l1_reg=lreg,
                scale_reg="inverse_group_size",
                subsampling_scheme=1,
                supress_warning=True,
            )

            gl.fit(X_train, y_train)
            cur_train_auc.append(predict_auc(X_train, y_train, gl))
            cur_val_auc.append(predict_auc(X_val, y_val, gl))
        total_train_auc = sum(cur_train_auc) / len(cur_val_auc)
        total_val_auc = sum(cur_val_auc) / len(cur_val_auc)
        
        train_auc_row.append(total_train_auc)
        val_auc_row.append(total_val_auc)
        print("Train AUC: {:.4f}".format(total_train_auc))
        print("Val AUC: {:.4f}".format(total_val_auc))

    train_auc.append(train_auc_row)
    val_auc.append(val_auc_row)


In [ ]:
train_auc = np.array(train_auc)
val_auc = np.array(val_auc)

In [ ]:
train_auc.tolist()

In [ ]:
val_auc

In [ ]:
l1_reg_lasso

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import matplotlib
import matplotlib as mpl


def make_heatmap(title, data, x_ticks, y_ticks):
    fig, ax = plt.subplots()
    im = ax.imshow(data)
    
    # Show all ticks and label them with the respective list entries
    ax.set_xticks(range(len(x_ticks)), labels=[f"{gl:.3f}" for gl in x_ticks],
                  rotation=45, ha="right", rotation_mode="anchor")
    ax.set_yticks(range(len(y_ticks)), labels=[f"{l1:.3f}" for l1 in y_ticks])
    
    
    
    train_auc = np.round(data, decimals=3)
    # Loop over data dimensions and create text annotations.
    for i in range(len(y_ticks)):
        for j in range(len(x_ticks)):
            text = ax.text(j, i, train_auc[i, j],
                           ha="center", va="center", color="w")
    
    ax.set_title(title)

    plt.xlabel("L1-Reg")
    plt.ylabel("Group Reg")
    plt.show()

make_heatmap("Train AUC", train_auc[::-1], l1_reg_lasso, group_reg_lesso[::-1])

In [ ]:
make_heatmap("Val AUC", val_auc[::-1], l1_reg_lasso, group_reg_lesso[::-1])

In [ ]:
max_index_flat = np.argmax(val_auc)


# Convert the flat index to a coordinate tuple
best_coords = np.unravel_index(max_index_flat, val_auc.shape)
print(f"Coordinates of max value: {best_coords}") # Output: (1, 0)
print(f"Maximum value: {val_auc[best_coords]}") # Output: 60

In [ ]:
best_gr = group_reg_lesso[best_coords[0]]
best_l1 = l1_reg_lasso[best_coords[1]]

print(f"Best Group Reg and L1-Reg: {(best_gr, best_l1)}")

In [ ]:
gl = LogisticGroupLasso(
    groups=groups,
    group_reg=best_gr,
    l1_reg=best_l1,
    scale_reg="inverse_group_size",
    subsampling_scheme=1,
    supress_warning=True,
)

gl.fit(train_X, train_y)

In [ ]:
group_train_AUC = predict_auc(train_X, train_y, gl)

In [ ]:
group_test_AUC = predict_auc(test_X, test_y, gl)

# Results

In [ ]:
print(f"Train AUC for Group Lasso: {group_train_AUC:.3f}")
print(f"Test AUC for Group Lasso: {group_test_AUC:.3f}")
print(f"Train AUC for Graphs NN: {final_graph_train_auc:.3f}")
print(f"Test AUC for Graph NN: {final_graph_test_auc:.3f}")

# All Groups

In [ ]:
[graph_list[i].name for i in range(get_num_graphs())]

# Chosen Groups for GraphNN

In [ ]:
graph_chosen = [graph_list[i].name for i,num in enumerate(final_coefs[0]) if abs(num) > 0]
graph_chosen.sort()

graph_chosen

In [ ]:
# number of chosen groups for gnn

len(graph_chosen)

# Chosen Groups for Group Lasso

In [ ]:
# make this show the names

chosen = [i.item() for i in gl.chosen_groups_]
chosen.sort()

[graph_list[c].name for c in chosen]

In [ ]:
chosen

In [ ]:
len(gl.chosen_groups_)